![logog](https://raw.githubusercontent.com/Pacific-AI-Corp/langtest/main/docs/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Pacific-AI-Corp/langtest/blob/main/demo/tutorials/misc/Upload_to_HuggingFace_Hub.ipynb)

**LangTest** is an open-source python library designed to help developers deliver safe and effective Natural Language Processing (NLP) models. Whether you are using **John Snow Labs, Hugging Face, Spacy** models or **OpenAI, Cohere, AI21, Hugging Face Inference API and Azure-OpenAI** based LLMs, it has got you covered. You can test any Named Entity Recognition (NER), Text Classification, fill-mask, Translation model using the library. We also support testing LLMS for Question-Answering, Summarization and text-generation tasks on benchmark datasets. The library supports 60+ out of the box tests. For a complete list of supported test categories, please refer to the [documentation](http://langtest.org/docs/pages/docs/test_categories).

Metrics are calculated by comparing the model's extractions in the original list of sentences against the extractions carried out in the noisy list of sentences. The original annotated labels are not used at any point, we are simply comparing the model against itself in a 2 settings.

# Getting started with LangTest on John Snow Labs

In [ ]:
!pip install "langtest[johnsnowlabs,transformers]" 

# Harness and its Parameters

The Harness class is a testing class for Natural Language Processing (NLP) models. It evaluates the performance of a NLP model on a given task using test data and generates a report with test results.Harness can be imported from the LangTest library in the following way.

In [7]:
#Import Harness from the LangTest library
from langtest import Harness

It imports the Harness class from within the module, that is designed to provide a blueprint or framework for conducting NLP testing, and that instances of the Harness class can be customized or configured for different testing scenarios or environments.

Here is a list of the different parameters that can be passed to the Harness function:

<br/>


| Parameter     | Description |
| - | - |
| **task**      | Task for which the model is to be evaluated (text-classification or ner) |
| **model**     | Specifies the model(s) to be evaluated. This parameter can be provided as either a dictionary or a list of dictionaries. Each dictionary should contain the following keys: <ul><li>model (mandatory): 	PipelineModel or path to a saved model or pretrained pipeline/model from hub.</li><li>hub (mandatory): Hub (library) to use in back-end for loading model from public models hub or from path</li></ul>|
| **data**      | The data to be used for evaluation. A dictionary providing flexibility and options for data sources. It should include the following keys: <ul><li>data_source (mandatory): The source of the data.</li><li>subset (optional): The subset of the data.</li><li>feature_column (optional): The column containing the features.</li><li>target_column (optional): The column containing the target labels.</li><li>split (optional): The data split to be used.</li><li>source (optional): Set to 'huggingface' when loading Hugging Face dataset.</li></ul> |
| **config**    | Configuration for the tests to be performed, specified in the form of a YAML file. |

<br/>
<br/>

# HuggingFace Dataset Augmentation for Text Classification

### Installing required dependencies

In [ ]:
!pip install datasets

### Setup and Configure Harness

In [41]:
harness = Harness(task="text-classification",
                  model={"model":"distilbert-base-uncased-finetuned-sst-2-english", "hub":"huggingface"},
                  data={"data_source":'glue',
                  "subset":"sst2",
                  "feature_column":"sentence",
                  "target_column":'label',
                  "split":"train",
                  "source": "huggingface"
                  })

Test Configuration : 
 {
 "tests": {
  "defaults": {
   "min_pass_rate": 1.0
  },
  "robustness": {
   "add_typo": {
    "min_pass_rate": 0.7
   },
   "american_to_british": {
    "min_pass_rate": 0.7
   }
  },
  "accuracy": {
   "min_micro_f1_score": {
    "min_score": 0.7
   }
  },
  "bias": {
   "replace_to_female_pronouns": {
    "min_pass_rate": 0.7
   },
   "replace_to_low_income_country": {
    "min_pass_rate": 0.7
   }
  },
  "fairness": {
   "min_gender_f1_score": {
    "min_score": 0.6
   }
  },
  "representation": {
   "min_label_representation_count": {
    "min_count": 50
   }
  }
 }
}


In [42]:
harness.configure(
{
 'tests': {'defaults': {'min_pass_rate': 0.65},
           'robustness': {'add_speech_to_text_typo':{'min_pass_rate': 0.60},
                          'add_ocr_typo':{'min_pass_rate': 0.60},
                        }
          }
 }
 )

{'tests': {'defaults': {'min_pass_rate': 0.65},
  'robustness': {'add_speech_to_text_typo': {'min_pass_rate': 0.6},
   'add_ocr_typo': {'min_pass_rate': 0.6}}}}

In [43]:
# Limit the data to the first 500 samples
harness.data = harness.data[:500]

### Generating the test cases

harness.generate() method automatically generates the test cases (based on the provided configuration)

In [44]:
harness.generate()

Generating testcases...: 100%|██████████| 1/1 [00:00<00:00, 348.05it/s]


### Running the tests

In [45]:
harness.run()

Running testcases... : 100%|██████████| 1000/1000 [02:05<00:00,  7.98it/s]


Called after harness.generate() and is to used to run all the tests. Returns a pass/fail flag for each test.

In [46]:
harness.generated_results()

,category,test_type,original,test_case,expected_result,actual_result,pass
0,robustness,add_speech_to_text_typo,hide new secretions from the parental units,hide new secretions frum the parental units',NEGATIVE,NEGATIVE,True
1,robustness,add_speech_to_text_typo,"contains no wit , only labored gags","contains know witte , only labored gags",NEGATIVE,NEGATIVE,True
2,robustness,add_speech_to_text_typo,that loves its characters and communicates som...,that loves its characters and communicates som...,POSITIVE,POSITIVE,True
3,robustness,add_speech_to_text_typo,remains utterly satisfied to remain the same t...,remains utterly satisfied to remain the sejm t...,NEGATIVE,NEGATIVE,True
4,robustness,add_speech_to_text_typo,on the worst revenge-of-the-nerds clichés the ...,aune the worst revenge-of-the-nerds clichés th...,NEGATIVE,NEGATIVE,True
...,...,...,...,...,...,...,...
995,robustness,add_ocr_typo,true star,trne ftar,POSITIVE,NEGATIVE,False
996,robustness,add_ocr_typo,"hampered -- no , paralyzed -- by a self-indulg...","hampered -- n^o , paralyzed -- by a self-indul...",NEGATIVE,NEGATIVE,True
997,robustness,add_ocr_typo,is expressly for idiots who do n't care what k...,is expressly f^r idiots avho do n't caie v\hat...,NEGATIVE,NEGATIVE,True
998,robustness,add_ocr_typo,is haunting ... ( it 's ) what punk rock music...,is haunting ... ( i^t 's ) v\hat punk rock mul...,POSITIVE,NEGATIVE,False


This method returns the generated results in the form of a pandas dataframe, which provides a convenient and easy-to-use format for working with the test results. You can use this method to quickly identify the test cases that failed and to determine where fixes are needed.

#### Report of the tests

In [47]:
harness.report()

,category,test_type,fail_count,pass_count,pass_rate,minimum_pass_rate,pass
0,robustness,add_speech_to_text_typo,35,465,93%,60%,True
1,robustness,add_ocr_typo,94,406,81%,60%,True


 Additional parameters (optional): You can pass additional parameters in the `training_data` dictionary to specify the details of the original dataset, such as the data source, subset, feature column, target column, and split. These parameters help in selecting the appropriate data for augmentation.

    - Example:
```
data_kwargs = {
    "data_source": "glue",
    "subset": "sst2",
    "feature_column": "sentence",
    "target_column": "label",
    "split": "train",
    "source": "huggingface"
}
```
        


In [ ]:
custom_proportions = {
    'add_ocr_typo':0.3
}

data_kwargs = {
      "data_source" : "glue",
      "subset": "sst2",
      "feature_column": "sentence",
      "target_column": "label",
      "split": "train",
      "source": "huggingface"
       }

harness.augment(
    training_data = data_kwargs,
    save_data_path ="augmented_glue.csv",
    custom_proportions=custom_proportions,
    export_mode="add",
)

Essentially it applies perturbations to the input data based on the recommendations from the harness reports. Then this augmented_dataset is used to retrain the original model so as to make the model more robust and improve its performance.

# Uploading Augmented Dataset to HuggingFace Hub

`Harness.upload_file_to_hub` function is a convenient utility provided by the Harness library for uploading **files/dataset** to the Hugging Face Model Hub repositories. This function enables users to easily share datasets, models, and other resources with the wider community through the Hugging Face ecosystem.

## Parameters

- `repo_name (str)`: The name of the repository on the Hugging Face Model Hub where the file will be uploaded. This should be in the format "Username/Repo-Name", where Username is the username of the user or organization on Hugging Face, and Repo-Name is the desired name of the repository.

- `repo_type (str)`: The type of repository to upload the file to. This can be "dataset", "model", or other supported repository types. In this context, it should be set to "dataset" if you want to upload a dataset.

- `file_path (str)`: The local file path of the file you want to upload. This should include the file name and its extension. For example, "augmented_glue.csv".

- `token (str)`: Your Hugging Face authentication token. This token is required for authentication and authorization to upload content to the specified repository. Make sure to keep your token confidential and secure.

- `path_in_repo (str)`: The desired path to the file within the repository. This includes the subdirectories and the filename. For instance, "data/augmented_glue.csv" indicates that the file should be placed in a data directory within the repository with the name augmented_glue.csv.

- `exist_ok` (bool) : If True, do not raise an error if repo already exists. Defaults to False.

## Usage
Here's an example of how to use the `Harness.upload_file_to_hub` function:

In [ ]:
Harness.upload_file_to_hub(repo_name="Username/Repo-Name",
                           repo_type="dataset", # If you want to upload a dataset
                           file_path="augmented_glue.csv",
                           token="hf_xxxxxxxxxxxxxxxxxx",
                           path_in_repo="data/augmented_glue.csv",
)

> Make sure to replace the placeholders (Username/Repo-Name, augmented_glue.csv, hf_xxxxxxxxxxxxxxxxxx, etc.) with your actual information.